# Text Feature Extraction

This document explains the basics of text vectorization using TFIDF. We are going to work on the [SMS Spam Dataset @ UCI](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) as a starter to check the basic ML pipeline with Scikit-Learn. The dataset labels an SMS text as `spam` or `ham`.

In order to improve the SMS classification or any similar NLP problem, we need to vectorize the text, i.e., convert words/tokens/strings into numerical values. One approach consists in creating a vector which contains an element for every unique/possible word in our texts: that generic vector to be filled in is a **vocabulary**. Each SMS/text is transformed into such a vector, and, in that form, it is considered to be a **bag of words**. With those bags of words, we can

- For each SMS or text document, count the number of times a word/term occurs; that way, we get the **Document Term Matrix**.
- A better alternative consists in weighting the term frequency in each text with the inverse frequency of appearance in all the texts: **TF-IDF, Term Frequency, Inverse Document Frequency**. This approach takes into account the importance of a word in the whole corpus, i.e., the complete text dataset. As such, stop words get a lower weight, thus less importance.

The TFIDF formulas are as follows (although everything is automatically computed):

- $\textrm{tfidf}(t,d,D) = \textrm{tf}(t,d) \cdot \textrm{idf}(t,D)$

- $\textrm{tf}(t,d) = \textrm{count}(t \in d)$: `count (term t in document d)`

- $\textrm{idf}(t,D) = \log \frac{N}{| \{d \in D \, : t \in d \} |}$: `log (total documents N / documents which contain term t)`

Note that the concept of bags of words and the vocabulary can be improved:
- Instead of words, we can used tokens that have been stemmed.
- In addition to counting the stemmed tokens, we can use additional information for each of them: morphological information (`pos_`) and syntactic information (`dep_`). Thus we end up having highly dimensional and sparse hypermatrices (tensors).

Overview of contents:

1. Manual Creation of Bags-of-Words
2. Text Feature Extraction with Scikit-Learn
   - 2.1 Load Dataset and Explore It
   - 2.2 Train/Test Split
   - 2.3 Vectorization
     - 2.3.1 Document Term Matrix: `CountVectorizer`
     - 2.3.2 Term Frequency Inverse Document Frequency: `TfidVectorizer`
   - 2.4 Model: Definition and Training
   - 2.5 Build a Pipeline
   - 2.6 Evaluate the Pipeline/Model
   - 2.7 Inference

*Diclaimer: I made this notebook while following the Udemy course [NLP - Natural Language Processing with Python](https://www.udemy.com/course/nlp-natural-language-processing-with-python/) by José Marcial Portilla. The original course notebooks and materials were provided with a download link, I haven't found a repository to fork from.*


## 1. Manual Creation of Bags-of-Words

This is a vanilla manual creation of vocabularies and bags or words. Not done in practice, since we use scikit-learn functionalities.

In [2]:
text1 = """This is a story about cats
        our feline pets
        Cats are furry animals
        """
text2 = """This story is about surfing
        Catching waves is fun
        Surfing is a popular water sport
        """

In [3]:
texts = [text1, text2]

In [5]:
# Build vocabulary (dictionary)
vocab = {}
i = 1
for text in texts:
    x = text.lower().split()
    for word in x:
        if word in vocab:
            continue
        else:
            vocab[word]=i
            i+=1

In [6]:
print(vocab)

{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12, 'surfing': 13, 'catching': 14, 'waves': 15, 'fun': 16, 'popular': 17, 'water': 18, 'sport': 19}


In [11]:
# Extract features
one = ['text1']+[0]*len(vocab)
two = ['text2']+[0]*len(vocab)
bow = [one, two]
i = 0
for text in texts:
    x = text.lower().split()
    for word in x:
        bow[i][vocab[word]] += 1
    i += 1

In [12]:
one

['text1', 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

In [13]:
two

['text2', 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1]

## 2. Text Feature Extraction with Scikit-Learn

### 2.1 Load Dataset and Explore It

In [14]:
import numpy as np
import pandas as pd

In [16]:
df = pd.read_csv('../data/smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [20]:
# Check missing values: None
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [21]:
# Check target: balanced? No
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

### 2.2 Train/Test Split

In [23]:
from sklearn.model_selection import train_test_split

X = df['message']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### 2.3 Vectorization

#### 2.3.1 Document Term Matrix: `CountVectorizer`

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
# Class that creates the Document Term Matrix
count_vect = CountVectorizer()
# Fit the class-object to the training split and transform the split
X_train_counts = count_vect.fit_transform(X_train)
# Vocabulary size: columns (number of features)
X_train_counts.shape

(3733, 7082)

#### 2.3.2 Term Frequency Inverse Document Frequency: `TfidVectorizer`

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Class that created the Document Term Matrix to be filled in with TFIDF values
vectorizer = TfidfVectorizer()
# Fit training split and transform it
X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

### 2.4 Model: Definition and Training

In [34]:
from sklearn.svm import LinearSVC
clf = LinearSVC() # classifier: clf
clf.fit(X_train_tfidf,y_train)

LinearSVC()

### 2.5 Build a Pipeline

Since in NLP all texts need to be pre-processed and vectorized, it is very common to create `Pipelines` in which we define all transformations required. Actually, since we need to deploy our application, every ML project should be packed into similar `Pipeline` objects.

In [36]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Now, we can pass the raw dataset
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

### 2.6 Evaluate the Pipeline/Model

In [37]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [41]:
# Report the confusion matrix
# It performs much better than before!
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [42]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [40]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


### 2.7 Inference

In [54]:
# HAM
text_clf.predict(["Hi, how are you doing?"])

array(['ham'], dtype=object)

In [55]:
# SPAM
text_clf.predict(["Congratulations! You've been selected as a winner. Send a message to 1-800-123-2345."])

array(['spam'], dtype=object)